In [56]:
from pyperplan import _parse, _ground
from argparse import ArgumentParser
import networkx as nx
import numpy as np

from scipy.stats import spearmanr

In [32]:
domain_path = "pddl_files/logistics/43/domain.pddl"
problem_path = "pddl_files/logistics/43/problogistics-6-1.pddl"
embedding_path = "logistics.emb"
edges_path = "logistics.edgelist"

In [33]:
# Parse embeddings
ordering = []
embeddings = []

with open(embedding_path, "r") as f:
    content = f.readlines()

for i in range(1, len(content)):
    split = content[i].split(" ")
    split[-1] = split[-1][:-2]

    ordering.append(split[0])
    embeddings.append(np.array(split[1:]))

ordering = np.array(ordering).astype(int)
embeddings = np.array(embeddings).astype(float)
new_embeddings = np.ones(embeddings.shape)
new_embeddings[ordering, :] = embeddings[np.arange(len(embeddings)),:]

In [51]:
# Create graph from edges
G = nx.Graph()

with open(edges_path, "r") as f:
    content = f.readlines()
    
for i in range(0, len(content)):
    split = content[i].split(" ")
    G.add_edge(int(split[0]), int(split[1][:-1]))

In [66]:
# Compare dot product to actual distance
dot_products = []
distances = []

for i in range(len(G.nodes)):
    for j in range(len(G.nodes)):
        if nx.has_path(G, i, j):
            dot_products.append(np.dot(new_embeddings[i], new_embeddings[j]))
            distances.append(nx.shortest_path_length(G, i, j))

In [65]:
nx.shortest_path_length(G, 0, 100)

5

In [67]:
spearmanr(dot_products, distances)

SpearmanrResult(correlation=-0.16714452830207727, pvalue=0.0)